## Motivation

The experiemental effort of loading single Na atom is proven to be quite challenging. Compare to other atoms that people have successfully trapped in a tweezer (Cs, Rb), we have identified many possible reasons including (not necessarily independent)

* Smaller exited state hyperfine structure
* Not as efficient sub-Doppler cooling
* No accessible magic wavelength for D2
* Stochastic heating

However, due to the characteristic of out system (Lamb-Dicke parameter $\eta\approx1$), it is very hard to analytically study these effects. On the other hand, since the system is very simple, a single atom with very few degrees of freedom, it is possible to do a precise numerical study of it.

## Methods

1. Quantum jump method

2. Split operator method


### Quantum jump method

The starting point of the simulation is the master equation

$$ \dot \rho=\frac1{i\hbar}[H,\rho]+\sum_mC_m\rho C_m^\dagger-\frac12\sum_m\left(C_m^\dagger C_m\rho +\rho C_m^\dagger C_m\right) $$

However, doing a deterministic calculation of the master equation requires the whole density matrix and many high dimensional matrix operations which is not very efficient. Instead, we use the [quantum jump method](https://en.wikipedia.org/wiki/Quantum_jump_method) (or Monte Carlo wave function (MCWF) method) to estimate the density matrix (and any measurable quantity derived from it) by evolving a wave function probabilistically.

* Rough scatch of the quantum jump method

    We divide the evolution into the coherent part described by the Hamiltonian $H$ and the incoherent part described by the jump operators $C_m$. In each time step of the simulation, we probabilistically choose either to evolve the wave function with the quantum jump or the Hamiltonian. The probability for doing each jump is determined by the expectation value of the jump operator $p_m=\langle C_m^\dagger C_m \rangle$.

    * If we decide to jump, the wave function is turned into the projection of $C_m$,

    $$ |\psi(t+\delta t)\rangle=C_m|\psi(t)\rangle $$

    * If we decide not to jump, the wave function is evolved with the original hamiltonian, plus a correction term due to the jump.

        $$ H'=H-\frac{i\hbar}{2}\sum_m C_m^\dagger C_m $$

    Since none of these operations are unitary, the wave function needs to be normalized at each time step.

Detail description of the method is available in thie master thesis http://www.oq.ulg.ac.be/master_thesis_rc.pdf

### Split operator method

The evolution with the Hamiltonian is done with the split operator method.

The Hamiltonian of the system can be expressed as

$$ H = H_x + H_p $$
  
where $H_x$ and $H_p$ are only functions of $x$ and $p$ respectively. The time evolution of the system is described by $e^{iH\delta t}$. This is hard to calculate since it is not diagonal in a good basis and the diagonalization is very time consuming.

So instead of directly calculation the exponential of the full Hamiltonian, we calculate the transformation $e^{iH_x\delta t}$ and $e^{iH_p\delta t}$ separately on the wave function. This is easy to do in the $x$ and $p$ basis and the tranformation of basis can be done very efficiently with [fast Fourier transformation](https://en.wikipedia.org/wiki/Fast_Fourier_transform)

## Performance optimization

1. Task parallelism

2. Reduce memory allocation

3. Memory locality
    
4. SIMD (Single instruction multiple data) (a.k.a. Data parallelism)

### Task parallelism

Since each time evolution in a Monte-Carlo simulation and each Monte-Carlo simulation in a multi-parameter scan are all independent, this is a embarrassingly parallelizable problem and can be done very efficiently with multiple processes (until the initialization cost kicks in).

### Reduce memory allocation

Memory allocations are expensive (to be improved). Reused buffers, use inplace operation, avoid boxing.

### Memory locality

The CPU is several orders of magnitude faster than the memory. Multiple levels of CPU Cache are introduced to hide this latency but they are much smaller than the main memory.
    
* It's very important to make sure the operations has very good memory locality to fully utilize the cache.
    
    This usually means the inner loop should iterate over the continuous dimension of an array so it is important to know how array are stored in memory.
        
    For example, in languages using row-major format (`C`/`C++`, `Mathematica`, `Python`) the preferred way to interate over an multi dimentional array is,
        
    ```c++
    for (int i = 0;i < n1;i++) {
        for (int j = 0;j < n2;j++) {
            A[i][j] = B[i][j] + C[i][j];
        }
    }
    ```
        
    in languages using column-major format (`MATLAB`, `R`, `Julia`), the preferred way is
        
    ```julia
    for i in 1:n1
        for j in 1:n2
            A[j, i] = B[j, i] + C[j, i]
        end
    end
    ```
        
    If it is necessary to iterate over both dimensions simultaneously (e.g. for matrix multiplication) it is sometimes beneficial to divide the array in to blocks to improve memory locality. See also [Locality of reference](https://en.wikipedia.org/wiki/Locality_of_reference#Matrix_multiplication).

* Predictable memory access patterns also help CPU prefetching.

* There are more to consider for multithreading

### SIMD (Single instruction multiple data) (a.k.a. Data parallelism)

Modern processors introduce instructions that can process multiple data at a time to speed up CPU bound data processing. Most popular ones on the x86 processors are the [SSE family](https://en.wikipedia.org/wiki/Streaming_SIMD_Extensions) and the more recent [AVX family](https://en.wikipedia.org/wiki/Advanced_Vector_Extensions). ARM processors also introduce [Advanced SIMD (NEON)](https://en.wikipedia.org/wiki/ARM_architecture#NEON) since ARMv6.

It is not always necessary to write assembly manually in order to exploit this function. The compiler can do the transformation automatically in many cases. However, this automatic optimization has some limitation

* Strided load

* Data dependency and alias analysis
